# Power Factory 

Group E

Project 3 Solution IPYNB

## Required Libraries and Setup

In [ ]:
import sys
sys.path.append('C:/Program Files/DIgSILENT/PowerFactory 15.2/Python/3.4')

In [ ]:
import powerfactory as pf
import pandas as pd
import datetime as dt
import os
import PythonMagick
from IPython.display import Image
import time
app = pf.GetApplication()
app.ActivateProject('LabPowerFactory')

## Exercise 2

## Necessary Classes

In [ ]:
# Load flow calculation class
load_flow_class = app.GetFromStudyCase('ComLdf')
# Time_object
study_time = app.GetFromStudyCase('SetTime')

## Get Objects

In [ ]:
subs=app.GetCalcRelevantObjects('*.ElmTrfstat')
lines=app.GetCalcRelevantObjects('*.ElmLne')
feeders=app.GetCalcRelevantObjects('*.ElmFeeder')

## Example
This is the code which should be executed after each time we change the configuration

In [ ]:

load_flow_class.Execute()

## Exercise 3

## To-Do: Build Incidence Matrix

### Example Code

In [ ]:
app.GetFromStudyCase('ComLdf').Execute()
layers = {}
net_layout = sorted(set([sub.GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot') for sub in subs]))
for l in net_layout:
    print(l)
    layers[l] = []
    for s in subs: 
        if s.GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot') == l:
            layers[l].append(s.loc_name)
layers



In [ ]:
#print(subs[2].loc_name)
#print(subs[2].GetAttribute('e:pBusbar'))
#subs[2].GetAttribute('e:pBusbar').GetAttribute('e:ciDistRoot')
lines[0].loc_name

## Radiality Check

In [ ]:
def radiality_check(feeders):
  radiality_is_met = True
  for feeder in feeders:
      is_radial = feeder.ciRadial
      if is_radial == 0:
          return (False, 'Grid is meshed')
  return(True, None)

In [ ]:
## just to check if the normal 
radiality_check(feeders)[0]   

## Feeded Check

In [ ]:
def feeded_check(subs): 
  all_are_fed = True
  for sub in subs:
      bus_bar = sub.GetAttribute('e:pBusbar')
      is_energized = bus_bar.IsEnergized()
      if is_energized == 0:
          return (False, sub.loc_name +' Is not energized')
  return(True, None)

In [ ]:
feeded_check(subs)[0]

## Graphic Save

This saves a graphic of grid and geolocation visualization. The name is given as input(figname) and the resulting images are under graphics/Exercise/RES

In [ ]:
project_folder_in_PF = app.GetProjectFolder('netmod')
graphic_board_folder = project_folder_in_PF.GetContents()[0].GetContents()


grid = [i for i in graphic_board_folder if 'Grid' in i.loc_name][0]
geo = [i for i in graphic_board_folder if 'Geographical' in i.loc_name][0]


def get_graphic(figname, boards=[grid,geo]):

    def project_new_folder(graph_path, grid_name,date_and_time,figname):
        new_path = graph_path+'/ '+grid_name
        if not os.path.exists(new_path):
            os.makedirs(new_path)
        graphic_object = app.GetFromStudyCase('SetDesktop')
        filename = os.path.join(new_path, date_and_time)
        res = graphic_object.WriteWMF(filename)
        img = PythonMagick.Image()
        img.density("125")
        img.read(filename + ".wmf")
        img.write(os.path.join(new_path, figname + "-"+ date_and_time + ".png")) # or .jpg

    #graph_path = r'C:\Users\.........'
    graph_path = r'graphics/Exercise/RES'
    project_name = app.GetActiveProject().loc_name
    new_path = graph_path +'/ '+ project_name
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    for key in boards:
        key.Show()
        this_graphic = key.loc_name
        graph_name = key.loc_name
        sim_dt_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
        usable_time = sim_dt_time.replace(':', '.')
        project_new_folder(new_path, graph_name , usable_time, figname)

In [ ]:
get_graphic("aha")